In [1]:
import os
from pathlib import Path

In [2]:
%pwd

'/home/utpal108/dev/Python/PW Skills Projects/Animal Detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/utpal108/dev/Python/PW Skills Projects/Animal Detection'

In [5]:
# Update the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from animalDetection.constants import *
from animalDetection.utils.common import  read_yaml, create_directories

In [7]:
# Update Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from animalDetection import logger
from animalDetection.utils.common import get_size

In [9]:
# Update Components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 


In [10]:
# Manage Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-07 14:26:30,426: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-07 14:26:30,427: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-07 14:26:30,428: INFO: common: created directory at: artifacts]
[2023-10-07 14:26:30,429: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-07 14:26:55,388: INFO: 2753422646: artifacts/data_ingestion/data.zip downloaded! with the following info: 
Connection: close
Content-Length: 9258107
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6ca329e467f2869fb23955f5bf0eb331e22fa8ca8df8c7dfc2a08bd1aca0f9ea"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 538A:1E2282:103CAF:144ED6:65211638
Accept-Ranges: bytes
Date: Sat, 07 Oct 2023 08:26:33 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1231-QPG
X-Cac